In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import cv2
from skimage.io import imread
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization


In [4]:
# Definimos el tamaño de la imagen (125x125) y los paths
IMAGE_SIZE = 125

TRAIN_PATH = "./data/movida/train/"

TEST_PATH = "./data/movida/test/"

In [5]:
# ------------------------------------------df_test---------------------------------------------

def read_data(path):
    archivos = []
    for file in os.listdir(path): 
            archivos.append(file)            
    return np.array(archivos)
    
archivos_test = read_data(TEST_PATH)

df_test = pd.DataFrame({
    'archivo': archivos_test})


# -----------------------------------------df_train-----------------------------------------------

df_train = pd.read_csv("./data/movida/file_target.csv")
  

In [6]:
print('train',df_train.head())
print('train shape',df_train.shape)
print('test',df_test.head())
print('test shape',df_test.shape)

train   archivo target
0   0.jpg      G
1   1.jpg      H
2   2.jpg      G
3   3.jpg      X
4   4.jpg      H
train shape (9000, 2)
test     archivo
0     0.jpg
1     1.jpg
2    10.jpg
3   100.jpg
4  1000.jpg
test shape (1150, 1)


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  9000 non-null   object
 1   target   9000 non-null   object
dtypes: object(2)
memory usage: 140.8+ KB


In [8]:
from keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,                                   
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [9]:
train_generator = train_datagen.flow_from_dataframe(df_train,
                                                    TRAIN_PATH,
                                                    x_col='archivo',
                                                    y_col='target',
                                                    batch_size = 20,
                                                    class_mode = 'sparse',
                                                    target_size = (IMAGE_SIZE, IMAGE_SIZE))


Found 9000 validated image filenames belonging to 29 classes.


In [ ]:
## VGG16

from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                  include_top=False)

for layer in base_model.layers:
    layer.trainable = False

    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='relu')(x)

# Add a fully connected layer with 512 hidden units and sigmoid activation
x = keras.layers.Dense(1000, activation='sigmoid')(x)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = keras.layers.Dense(29, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = model.fit(train_generator,
                    epochs = 35)

In [ ]:
vgghist = model.fit(train_generator,
                    epochs = 10)

In [11]:
## EfficientNetB7

import tensorflow.keras.applications as apli
from tensorflow.keras.optimizers import RMSprop

base_model = apli.EfficientNetB7(input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                                 include_top=True,
                                 weights=None)

'''for layer in base_model.layers:
    layer.trainable = False

    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='relu')(x)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='sigmoid')(x)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = keras.layers.Dense(1000, activation='relu')(x)

# Add a dropout rate of 0.5
x = keras.layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = keras.layers.Dense(29, activation='softmax')(x)'''

#modelefi = tf.keras.models.Model(base_model.input)

base_model.compile(RMSprop(lr=0.0001, decay=1e-6), loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [12]:
efihist = base_model.fit(train_generator, steps_per_epoch = 100,
                    epochs = 10)

Epoch 1/10
 13/100 [==>...........................] - ETA: 28:11 - loss: 6.9317 - acc: 0.0032

KeyboardInterrupt: 

In [ ]:
predictions = base_model.predict(X_test_norma)

In [ ]:
model.evaluate(X_test_norma)

In [ ]:
predictions = model.predict(X_test_norma)

In [ ]:
predict = predictions.argmax(axis=1)

predict

In [ ]:
import collections

collections.Counter(predict)

In [ ]:
df_test['target']=predict

In [ ]:
series = {}

for i in range(0,29):
    A = df_train[df_train['target_2']==i]['target'].head(1)
    B =A.values
    series[B[0]]=i

In [ ]:
list(series.keys())[list(series.values()).index(0)]

In [ ]:
df_test_2 = df_test

In [ ]:
def change_number(x):

    return list(series.keys())[list(series.values()).index(x)]

df_test_2['target'] = df_test_2['target'].apply(change_number)


In [ ]:
df_test_2

In [ ]:
df_test_2.to_csv('data/submission.csv', index=False)